<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/Time%20Series/stock_algo_trading_CCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import math
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 9)   # Change the size of plots

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# !pip yfinance
import yfinance as yf

# https://github.com/atreadw1492/yahoo_fin
# !pip install yahoo_fin
from yahoo_fin.stock_info import * # get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table 

# !pip install ta
import ta 

In [2]:
# get_quote_table('aapl')
# get_data('aapl')

In [3]:
start = datetime.datetime(2016,1,1)
# start = datetime.datetime(2020,3,21)
end = datetime.date.today()

Billion = 1000000000  
Million = 1000000  

def get_marketcap(symbol):
    tmp = get_quote_table(symbol)['Market Cap']
    if 'B' in tmp:
        return  Billion * float(tmp[:-1])
    elif 'M' in tmp:
        return  Million * float(tmp[:-1])
    else:
        print(f'What????, {tmp}')

In [4]:
# Correct as of 18 may https://www.blackrock.com/ca/investors/en/products/239839/ishares-sptsx-capped-energy-index-etf
symbols = ['CNQ.to', 'SU.to', 'CVE.to', 'IMO.to', 'TOU.to', 'ARX.to', 'WCP.to', 'PSK.to', 'CPG.to', 'PXT.to', 'MEG.to', 'ERF.to', 'VET.to']
weights = [25.51, 24.45, 11.96, 8.41, 7.67, 5.90, 2.80, 2.69, 2.56, 2.55, 2.08, 1.89, 1.50] # 0.03 in cash 
weights = np.array(weights)/100 
symbol_weights = dict(zip(symbols, weights))

In [13]:
# dataset = dict()
# dataset['XEG.TO'] = yf.download('XEG.TO', start, end)["Adj Close"]

computed_price = 0.03/100 # cash held     
computed_price_alt = 0.03/100
for symbol in symbols:
#     dataset[symbol] = yf.download(symbol, start, end)["Adj Close"]
   
    marketcap = get_marketcap(symbol)
    price = get_live_price(symbol)
    weight = symbol_weights[symbol]
    
    price_of_this =  0 # or price *weight
    computed_price += (weight * marketcap) / price
    computed_price_alt += weight*price
    print(f'{symbol}\t Weighted price is {price:.2f} * {weight:.2f} = {weight * price:.2f}')
    
# stocks = pd.DataFrame(dataset)
print(f'\nComputed price {computed_price:.2f} Expected price {get_live_price("XEG.to"):.2f}. computed_price_alt {computed_price_alt:.2f}')

CNQ.to	 Weighted price is 41.86 * 0.26 = 10.68
SU.to	 Weighted price is 28.79 * 0.24 = 7.04
CVE.to	 Weighted price is 9.91 * 0.12 = 1.19
IMO.to	 Weighted price is 39.33 * 0.08 = 3.31
TOU.to	 Weighted price is 29.25 * 0.08 = 2.24
ARX.to	 Weighted price is 9.31 * 0.06 = 0.55
WCP.to	 Weighted price is 5.86 * 0.03 = 0.16
PSK.to	 Weighted price is 13.77 * 0.03 = 0.37
CPG.to	 Weighted price is 5.01 * 0.03 = 0.13
PXT.to	 Weighted price is 20.34 * 0.03 = 0.52
MEG.to	 Weighted price is 7.68 * 0.02 = 0.16
ERF.to	 Weighted price is 7.66 * 0.02 = 0.14
VET.to	 Weighted price is 9.61 * 0.01 = 0.14

Computed price 1091599160.37 Expected price 8.31. computed_price_alt 26.63


#### XEG.CA Split History:    Aug 6, 2008 - 4:1

> I didnt know weights were of market cap

In [22]:
# computed_price_adj = computed_price/4
computed_price__alt_adj = computed_price_alt/4

print(f'Computed price {computed_price:.2f} Expected price {get_live_price("XEG.to"):.2f}. computed_price_alt {computed_price__alt_adj:.2f}')

Computed price 1091599160.37 Expected price 8.31. computed_price_alt 6.66


In [23]:
XEG_shares_outstanding = 152500000
computed_price/XEG_shares_outstanding

computed_price__alt_adj

7.158027281111992

6.6584206385612505

## Another way...
prices * norm(weights*marketcaps)

In [8]:
myList = [symbol_weights[symbol] * get_marketcap(symbol) for symbol in symbols]
# To correctly normalize
myList.append(358250.70) # * 0.03/100 # cash, 

myList
weights_times_marketcap_normalized = np.array(myList)/sum(myList)
sum(weights_times_marketcap_normalized)
weights_times_marketcap_normalized

[12652194700.0,
 10604454000.0,
 2391162800.0,
 2428051100.0,
 666139500.0,
 397542000.0,
 97999999.99999999,
 82717500.0,
 74598400.0,
 65483999.99999999,
 45884800.0,
 37176300.0,
 22965000.0,
 358250.7]

1.0

array([4.27920010e-01, 3.58661732e-01, 8.08734322e-02, 8.21210609e-02,
       2.25300376e-02, 1.34455864e-02, 3.31453649e-03, 2.79765482e-03,
       2.52305223e-03, 2.21478681e-03, 1.55190657e-03, 1.25736942e-03,
       7.76717658e-04, 1.21166839e-05])

In [9]:
summer = (0.03/100)
for i in range(len(symbols)): 
    summer += weights_times_marketcap_normalized[i] * get_live_price(symbols[i])
summer/4    

8.299753292873625

In [10]:
# stocks.head(5)
# stocks.tail(5)

In [11]:
# stocks.shape[0]
# stocks=stocks.dropna()
# stocks.shape[0]
# stocks.plot(grid=True, figsize=(18, 6))

In [12]:
# def normIt(myDf):
#    return (myDf - myDf.mean()) / (myDf.max() - myDf.min())

# stocks_norm = normIt(stocks)
# stocks_norm.shape[0]
# stocks_norm= stocks_norm.dropna()
# stocks_norm.shape[0]
# stocks_norm.plot(grid=True, figsize=(18, 6))